In [17]:
import time
import paramiko
from io import StringIO
import time
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output
import datetime

In [2]:
# SSH credentials and remote CSV file path
host = 'lxslc7.ihep.ac.cn'
username = 'frank'
password = '229221518fF!'
remote_csv_file = '/hpcfs/cepc/higgs/frank/GNN/quick_test/output/summaries_0.csv'

# Establish an SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, username=username, password=password)

In [20]:
def update_csv():
    sftp = ssh.open_sftp()
    with sftp.open(remote_csv_file, 'r') as file:
        remote_csv_data = file.read().decode('utf-8')
        timestamp = sftp.stat(remote_csv_file).st_mtime

        dt_object = datetime.datetime.fromtimestamp(timestamp)

        # Parse and display the CSV data
        df = pd.read_csv(StringIO(remote_csv_data))
        return df, f"{dt_object.year} - {dt_object.month} - {dt_object.day} --> {dt_object.hour} : {dt_object.minute}"


def update_fig(df, t):
    fig.data = []
    fig.add_trace(go.Scatter(x=df['epoch'], y=df['train_loss'], mode='lines+markers'))
    fig.add_trace(go.Scatter(x=df['epoch'], y=df['valid_loss'], mode='lines+markers'))
    fig.update_layout(
        title=t
    )

In [21]:
fig = go.FigureWidget()

df, t = update_csv()
update_fig(df, t)
fig.show()
# while True:
#     df = update_csv()
#     update_fig(df)
#     time.sleep(15)
#     clear_output(wait=True)
#     fig.show()

2023 - 3 - 24 --> 17 : 22
